<h2>4.全局内存</h2>
<h3>4.1CUDA内存模型概述</h3>
<p>GPU和CPU都使用类似的内存层次结构设计，主要区别在于，CUDA编程模型能将内存层次结构更好地呈现给用户，能让使用者更好的控制它的行为</p>
<h3>4.1.2内存模型</h3>
<p>与CPU模型不可对一级缓存和二级缓存编程不同，CUDA内存模型提出多种可编程模型：</p>
<p>寄存器 共享内存 本地内存 常量内存 纹理内存 全局内存</p>
<p>线程中含私有本地内存 线程块中含共享内存（块中线程共享） 所有线程均可访问全局内存 所有线程可访问只读内存空间（常量内存空间 纹理内存空间） 纹理内存为各种数据布局提供不同的寻找模式 全局/常量/纹理有相同的生命周期</p>

<h4>4.1.2.1寄存器</h4>
<p>速度最快 自变量 索引为常量的数组 私有 生命周期与核函数相同</p>

<h4>4.1.2.2本地内存</h4>
<p>寄存器溢出的进入本地内存 未知索引引用的本地数组 较大本地结构体或数组 变量</p>

<h4>4.1.2.3共享内存</h4>
<p>__shared__</p>
<p>片上内存->可编程的一级缓存 比本地内存和全局内存更高带宽和更低延迟 生命周期为线程块 线程之间通信的基本方式</p>
<p>访问时需调用 void _syncthreads() 避免数据冲突</p>

<h4>4.1.2.4常量内存</h4>
<p>__constant__</p>
<p>在全局内存和所有核函数外声明，设备只可声明64KB</p>
<p>核函数只能从常量内存中读取数据 只能在主机端初始化</p>
<p>用于线程束中所有线程从相同内存地址读取数据</p>

<h4>4.1.2.5纹理内存</h4>
<p>设备内存中的内存， 一种通过指定的只读缓存访问的全局内存</p>

<h4>4.1.2.6全局内存</h4>
<p>GPU中最大/延迟最高且最长使用的内存</p>
<p>__device__静态声明变量 cudaMalloc cudaFree动态分配释放</p>

<h4>4.1.2.7 GPU缓存</h4>
<p>一级缓存（每个SM） 二级缓存（所有SM） 只读常量缓存（每个SM） 只读纹理缓存（每个SM）</p>

<h3>4.2内存模型</h3>
<h4>4.2.1 内存分配和释放</h4>
<p>分配 cudaError_t cudaMalloc(void **devPtr, size_t count);</p>
<p>填充 cudaMemset 释放 cudaFree</p>
<h4>4.2.2 内存传输</h4>
<p>cudaMemcpy 尽可能减少主机和设备之间内存传输</p>
<h4>4.2.3 固定内存</h4>
<p>cudaMallocHost 在host无需从分页内存转为固定内存</p>
<h4>4.2.4 零拷贝内存</h4>
<p>主机和设备均能访问</p>
<p>设备内存不足时，避免主机设备显式数据传输，提高PCIe传输率 为固定内存</p>
<h4>4.2.5 统一虚拟寻址</h4>
<p>UVA 主机内存和设备内存可以共享同一个虚拟内存空间</p>
<h4>统一内存寻址</h4>
<p>创建一个托管内存池，内存池中已分配空间可以用相同的内存地址在CPU和GPU上访问 对应用程序透明</p>
<p>__managed__->托管变量 可从主机或设备代码中被引用
cudaMallocManaged</p>
<h3>4.3内存访问模式</h4>
<h4>4.3.1对齐与合并访问<h4>
<p>全局内存为逻辑地址空间，可通过核函数访问。核函数内存请求通常在DRAM设备和片上内存间以128字节或32字节内存事物实现</p>
<p>所有对全局内存访问会通过二级缓存，也有许多通过一级缓存。128字节内存访问用到两级缓存，32字节内存访问只用二级缓存</p>
<p>一行以及缓存为128字节，映射到内存中有一个128字节对齐段，因此注意设备访问两个特性：对齐内存访问/合并内存访问</p>
<p>事物服务缓存粒度偶数倍出现对齐内存访问，非对齐会造成带宽浪费</p>
<p>一个线程束中32个线程访问连续的一个内存块时，出现合并内存访问</p>
<h4>4.3.2全局内存读取</h4>
<p>SM中数据通过3种缓存/缓冲路径传输：1/2级缓存（默认） 常量缓存 只读缓存</p>
<h5>4.3.2.1缓存加载</h5>
<p>5种情况: 1 SM请求对齐的连续128字节，一次事物 总线利用率100% 2 SM请求对齐的随机128字节，一次事物 总线利用率100% 3 SM请求非对齐连续128字节 总线利用率50% 4 SM所有线程请求相同地址 一次事物 总线利用率4/128 = 3.125% 5 线程束请求分散于全局内存中32个4字节地址，占用N个缓存行，需N次内存事物</p>
<p>CPU->空间时间局部性 GPU->空间局部性</p>
<h5>4.3.2.2没有缓存的加载</h5>
<p>不经过一级缓存 在内存段粒度上执行(32个字节)->更好的总线利用率</p>
<h5>4.3.2.3非对齐读取示例</h5>
<h5>4.3.2.4只读缓存</h5>
<p>只读缓存两种方式读取 __lgd 间接引用指针修饰符</p>

<h4>4.3.3 全局内存写入</h4>
<p>存储只能通过二级缓存（32字节粒度）</p>

<h4>4.3.4结构体数组(SoA)与数组结构体(AoS)</h4>
<p>AoS在GPU上只执行部分字段的应用程序将导致带宽损失 同时在未执行字段浪费二级缓存</p>
<p>SoA充分利用带宽，无交叉元素</p>

<h4>4.3.5性能调整</h4>
<h5>4.3.5.1展开技术</h5>
<p>展开对性能有非常好的影响，比地址对齐要好，展开不影响内存操作的数量，只影响并发执行的数量</p>
<p>最大化带宽利用率：1 有效利用设备DRAM和SM片上内存之间的字节移动，内存访问对齐合并 2 最大化当前存储器操作数 (1)展开 每个线程更多独立内存访问 (2)修改核函数启动配置使每个SM更多并行性</p>

<h3>4.4 核函数可达到的带宽</h3>
<h4>4.4.1 内存带宽</h4>
<p>矩阵转置 交叉访问？</p>
<p>按列读取 按行存储 瘦块</p>